In [9]:
import openrouteservice as ors
import folium

start = []
end = []
coords = [[8.685207, 49.37717], [8.685207, 49.419349]]
host = "http://localhost:8082/ors"
profile = "driving-car"
formats = "geojson"

client = ors.Client(base_url=host)
resp = client.request(url = f'/v2/directions/{profile}/geojson', get_params={}, post_json = {'coordinates': coords })


# create map centered on Heidelberg
m = folium.Map(location = [49.411085, 8.685894], zoom_start = 13)
folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           resp['features'][0]['geometry']['coordinates']]).add_to(m)
m